In [1]:
from langgraph.graph import StateGraph, START, END
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from typing import TypedDict
from dotenv import load_dotenv
import os


d:\New folder (2)\Agentic\agenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
load_dotenv()

True

In [8]:
# Step 1: Create HuggingFaceEndpoint instance
llm = HuggingFaceEndpoint(
    repo_id="openai/gpt-oss-120b",  # ✅ Chat-compatible model
    task="text-generation",
    huggingfacehub_api_token=os.getenv("HUGGINGFACEHUB_API_TOKEN")
)

# Step 2: Pass it into ChatHuggingFace
model = ChatHuggingFace(llm=llm)

In [9]:
# create a state

class LLMState(TypedDict):
    question: str
    answer: str

In [10]:
def llm_qa(state: LLMState) -> LLMState:

    # extract the question from state
    question = state ['question']

    # from a prompt
    prompt = f'Answer the following question {question}'

    # ask that question to the LLM
    answer = model.invoke(prompt).content

    # update the answer in the state 
    state['answer'] = answer

    return state

In [11]:
# create our graph
graph = StateGraph(LLMState)

# add nodes
graph.add_node('llm_qa', llm_qa)

# add edges
graph.add_edge(START, 'llm_qa')
graph.add_edge('llm_qa', END)

# compile
workflow = graph.compile()

In [13]:
# execute

intial_state = {'question': 'How far is moon from the earth?'}

final_state = workflow.invoke(intial_state)

print(final_state['answer'])

The Moon orbits the Earth at an average distance of **about 384,400 kilometers (≈238,900 miles)**.

Because the Moon’s orbit is elliptical, this distance varies:

| Position | Approximate distance from Earth |
|----------|---------------------------------|
| **Perigee** (closest point) | ~363,300 km (≈225,700 mi) |
| **Apogee** (farthest point) | ~405,500 km (≈252,000 mi) |
| **Average (mean) distance** | ~384,400 km (≈238,900 mi) |

These figures are based on the geocentric (Earth‑centered) distance to the Moon’s center. If you measure from the surface of the Earth to the surface of the Moon, subtract the Earth's radius (~6,371 km) and add the Moon’s radius (~1,737 km), which changes the numbers by only a few thousand kilometers.
